# Used Cars from eBay Kleinanzeigen

An analysis and cleaning of used car data originally uploaded to Kaggle, now available via Dataquest. The source used in this analysis is a 50k data point sampling from the original copy.

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

### Initial View of Data

There are 20 columns and 50k entries total. Some columns, such as `yearOfRegistration`, `powerPS`, `monthOfRegistration`, `nrOfPictures`, and `postalCode` are int types. 

However, columns such as `price` and `odometer` need to be converted to float and int types respectively.

Additionally some columns have null values, and column names use camelcase.

In [2]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [3]:
autos.columns


Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

## Reassigning Column Names

First we will rename the columns so they conform to Python naming standards, EG snakecase, and are more descriptive. 

In [4]:
autos.columns = [ 'date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'powerPS', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postalCode', 'last_seen']

In [5]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postalCode,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Reviewing Summary Metrics (Pre-Cleanse)

Upon initial review, we should drop `nr_of_pictures` as all values are 0. `seller` and `offer_type` have mainly the same values; additionally,`registration_year` also needs review as the minimum year is 1000 and maximum is 9999.

`price` and `odometer` need to be cleaned and converted to numerical types so that these fields may be reviewed as well.

In [6]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postalCode,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


## Cleaning Price and Odometer KM columns

We remove special characters from the fields and convert `price` to float type and `odometer` to int type.

In [7]:
autos["price"] = autos["price"].str.replace("$","").str.replace(",","").astype('float')

In [8]:
autos["odometer_km"] = autos["odometer_km"].str.replace("km","").str.replace(",","").astype('int')

In [9]:
autos["registration_year"].value_counts()

2000    3354
2005    3015
1999    3000
2004    2737
2003    2727
        ... 
1931       1
1929       1
1001       1
9996       1
1952       1
Name: registration_year, Length: 97, dtype: int64

## Odometer and Price Cleaning and Observations

### Reviewing the Odometer Column

There are 13 values total and the majority of cars have odometers at 150,000, the maximum number. No need to remove outliers. 

Since there are few values available it is possible that sellers select odometer intervals when putting their vehicle up for sale.

In [10]:
autos["odometer_km"].unique().shape

(13,)

In [11]:
autos["odometer_km"].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [12]:
autos["odometer_km"].value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

### Reviewing the Price Column

There are 2351 unique values. 1421 cars have a price of 0.00, which doesn't seem right for a bidding website.

So, we explore the top and bottom rung values for `price` and remove outliers. We require that the value should be at least 1 with a ceiling of 350000.0. Thus, we are removing values of 1M+, of which there are a few.

In [13]:
autos["price"].unique().shape

(2357,)

In [14]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [15]:
autos["price"].value_counts().sort_index(ascending=True).head(25)

0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0       3
18.0       1
20.0       4
25.0       5
29.0       1
30.0       7
35.0       1
40.0       6
45.0       4
47.0       1
49.0       4
50.0      49
Name: price, dtype: int64

In [16]:
autos["price"].value_counts().sort_index(ascending=False).head(25)

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
259000.0      1
250000.0      1
220000.0      1
198000.0      1
197000.0      1
194000.0      1
190000.0      1
180000.0      1
175000.0      1
169999.0      1
Name: price, dtype: int64

In [17]:
autos = autos[autos["price"].between(1,350000.0)]

In [18]:
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

## Date Field Observations

There are 3 date fields in the data set, which as timestamps logged as strings. We will observe each timestamp as a date with frequency percentages.

### Date Crawled 

Dates from 3/20/2016-4/2/2016, which are the earliest available dates, account for 4% (rounded up) of values each. 

The most recent dates, 4/6-4/7/2016, have the least representation at less than 1% each.

In [19]:
print(autos["date_crawled"].str[:10].value_counts(normalize=True, dropna=False).sort_index)

<bound method Series.sort_index of 2016-04-03    0.038608
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-12    0.036920
2016-03-14    0.036549
2016-04-04    0.036487
2016-03-07    0.036014
2016-04-02    0.035478
2016-03-28    0.034860
2016-03-19    0.034778
2016-03-15    0.034284
2016-03-29    0.034099
2016-04-01    0.033687
2016-03-30    0.033687
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-22    0.032987
2016-03-11    0.032575
2016-03-23    0.032225
2016-03-26    0.032204
2016-03-10    0.032184
2016-03-31    0.031834
2016-03-17    0.031628
2016-03-25    0.031607
2016-03-27    0.031092
2016-03-16    0.029610
2016-03-24    0.029342
2016-03-05    0.025327
2016-03-13    0.015670
2016-03-06    0.014043
2016-04-05    0.013096
2016-03-18    0.012911
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64>


### Ad Created

There are 76 dates total ranging from 6/11/2015-4/7/2016, and the earliest dates have the least representation in the set. The latest dates have 4% and 1% representation.

In [20]:
print(autos["ad_created"].str[:10].value_counts(normalize=True, dropna=False).sort_index())

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64


### Last Seen

The dates range from 3/5/2016-4/7/2016. 22% of records have a last seen date of 4/6, 12% on 4/5, and 13% on 4/7. Dates from 3/20-4/4 account for 2% of records each.

In [21]:
print(autos["last_seen"].str[:10].value_counts(normalize=True, dropna=False).sort_index())

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64


## Cleaning Registration Years

The registration year is inaccurate for some records as the minimum is 1000 and the max is 9999.

When we create a filter for records before 1900, we see some odd years (EG 1800, 1000), so we will eliminate records prior 1900. We have observed the highest `last seen` date as 4/7/2016 so we will set our upper limit to 2016.

With our newly restricted records observing registration years 1900-2016, we now have a mean registration year of 2002 with the 50th percentile at 2003. We see that 7% of records have a registration year of 2000 and 6% each for 2005 and 1999.

In [22]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [23]:
lowyear = autos["registration_year"] < 1910

In [24]:
print(autos[lowyear])

              date_crawled                                   name  seller  \
10556  2016-04-01 06:02:10                             UNFAL_Auto  privat   
22316  2016-03-29 16:56:41  VW_Kaefer.__Zwei_zum_Preis_von_einem.  privat   
24511  2016-03-17 19:45:11            Trabant__wartburg__Ostalgie  privat   
32585  2016-04-02 16:56:39                             UNFAL_Auto  privat   
49283  2016-03-15 18:38:53                             Citroen_HY  privat   

      offer_type   price   abtest vehicle_type  registration_year  gearbox  \
10556    Angebot   450.0  control          NaN               1800      NaN   
22316    Angebot  1500.0  control          NaN               1000  manuell   
24511    Angebot   490.0  control          NaN               1111      NaN   
32585    Angebot   450.0  control          NaN               1800      NaN   
49283    Angebot  7750.0  control          NaN               1001      NaN   

       powerPS   model  odometer_km  registration_month fuel_type   

In [25]:
autos = autos[autos["registration_year"].between(1900,2016)]

In [26]:
autos["registration_year"].describe()

count    46681.000000
mean      2002.910756
std          7.185103
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

In [27]:
autos["registration_year"].value_counts(normalize=True)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
          ...   
1938    0.000021
1948    0.000021
1927    0.000021
1931    0.000021
1952    0.000021
Name: registration_year, Length: 78, dtype: float64

## Observing Brands

According to initial distribution analysis, 20% of vehicles are Volkswagen, 11% BMW, and 10.7% Opel. We will review these plus Mercedes Benz, Audi, and Ford as these brands make up 6% or more of records each.

We loop through a list of these brands to calculate mean prices for each.

In [28]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

In [29]:
brands = ["volkswagen","bmw","opel","mercedes_benz","audi","ford"]
brand_agg = {}
for b in brands:
    brand = autos[autos["brand"]==b] #apply filter to series
    mean_price = brand["price"].mean() #calculate mean price
    brand_agg[b]=mean_price #assign to dictionary

## Average Prices by Popular Brand

Audis and Mercedes Benz sell for the most at 9336 and 8628, while Opels and Fords sell for the least at 2975 and 3749. Volkswagens are middling offerings at 5402.

In [30]:
print(brand_agg)

{'volkswagen': 5402.410261610221, 'bmw': 8332.820517811953, 'opel': 2975.2419354838707, 'mercedes_benz': 8628.450366422385, 'audi': 9336.687453600594, 'ford': 3749.4695065890287}


## Mileage against Price in Popular Brands

To compare mileage against mean prices for these brands, we will create another dictionary for mileage using the `odometer_km` column then merge the two dictionaries into a dataframe.

It appears that most brands range from 12.8-13.2K on odometer readings, excluding Ford which stands out as the cheapest and having the lowest average odometer readings at 12.4k.

In [31]:
brand_km = {}

for b in brands:
    brand = autos[autos["brand"]==b] ##filter for brand
    mileage = brand["odometer_km"].mean() ##calculate mean odometer
    brand_km[b] = mileage ##assign to dictionary

print(brand_km)

{'volkswagen': 128707.15879132022, 'bmw': 132572.51313996495, 'opel': 129310.0358422939, 'mercedes_benz': 130788.36331334666, 'audi': 129157.38678544914, 'ford': 124266.01287159056}


In [32]:
brand_series = pd.Series(brand_agg)
brand_odom = pd.Series(brand_km)

In [37]:
df = pd.DataFrame(brand_series, columns = ['mean_price'])

In [38]:
display(df)

,mean_price
volkswagen,5402.410262
bmw,8332.820518
opel,2975.241935
mercedes_benz,8628.450366
audi,9336.687454
ford,3749.469507


In [39]:
df["mean_odom"] = brand_odom

In [40]:
display(df)

,mean_price,mean_odom
volkswagen,5402.410262,128707.158791
bmw,8332.820518,132572.513140
opel,2975.241935,129310.035842
mercedes_benz,8628.450366,130788.363313
audi,9336.687454,129157.386785
ford,3749.469507,124266.012872
